## Converting to pytorch lightning fitting routine 

In [1]:
!export CUDA_LAUNCH_BLOCKING=1

In [1]:
# import tensorboard
import torch
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl


VOCAB_SIZE = 32128

class Routine(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.lr = 1e-3
        self.validation_step_outputs = []
        self.training_step_outputs = []
        self.test_step_outputs = []

    def forward(
        self,
        input_ids,
        attention_mask,
        labels,
        decoder_attention_mask, 
    ):
        y_hat = self.model(input_ids=input_ids, attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask, labels=labels)
        
        # print(f"forward(): {y_hat=}")
        return y_hat

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # print(f"keys = {batch.keys()}")
        # print(f"{batch=}")
        y = batch['labels']
        y_hat = self(**batch)
        y_onehot = F.one_hot(y, num_classes=VOCAB_SIZE)
        y = y_onehot.float()
        losses = []
        # computing cross-entropy on per-token basis and averaging the loss. 
        for tok in range(y_hat.logits.shape[1]):
            # print("Per-token loss cross entropy")
            loss = F.cross_entropy(y_hat.logits[:,tok,:] , y[:,tok,:])
            # print(loss)
            # loss = F.nll_loss(y_hat[:,tok,:] , y[:,tok,:])
            losses.append(loss)

        loss  = torch.tensor(losses,  requires_grad=True).mean()
        # dummy metrics


        # calculating exact matches 
        y_hat = F.softmax(y_hat.logits, dim=-1)
        a = y_hat.argmax(1)
        y_hat = torch.zeros(y_hat.shape).scatter(1, a.unsqueeze(1), 1.0)
        # y_onehot = F.one_hot(y, num_classes=VOCAB_SIZE)
        # y = y_onehot.float()
        matches = (y == y_hat).int()
        correct = matches.sum()
        tot = torch.prod(torch.tensor(matches.shape))
        metrics_dict = {"loss": loss, "train_EM": (correct/tot).item(), "train_F1": 0.9}
        # print(metrics_dict)
        self.training_step_outputs.append(metrics_dict)
        return metrics_dict



    def on_train_epoch_end(self):
        results = {
            "loss": torch.tensor(
                [x["loss"] for x in self.training_step_outputs]
            ).mean(),
            "F1": torch.tensor(
                [x["train_F1"] for x in self.training_step_outputs]
            ).mean(),
            "EM": torch.tensor(
                [x["train_EM"] for x in self.training_step_outputs]
            ).mean(),
        }
        # self.log(f"LR",self.lr, on_epoch=True, prog_bar=True, logger=True)
        for k, v in results.items():
            self.log(
                f"train_{k}",
                v,
                on_epoch=True,
                prog_bar=True,
                logger=True,
                sync_dist=True,
            )

    def validation_step(self, batch, batch_idx):
        y = batch['labels']
        y_hat = self(**batch)
        y_onehot = F.one_hot(y, num_classes=VOCAB_SIZE)
        y = y_onehot.float()
        losses = []
        # computing cross-entropy on per-token basis and averaging the loss. 
        for tok in range(y_hat.logits.shape[1]):
            loss = F.cross_entropy(y_hat.logits[:,tok,:] , y[:,tok,:])
            # print(loss)
            losses.append(loss)
        loss  = torch.tensor(losses).mean()



        # calculating exact matches 
        y_hat = F.softmax(y_hat.logits, dim=-1)
        a = y_hat.argmax(1)
        y_hat = torch.zeros(y_hat.shape).scatter(1, a.unsqueeze(1), 1.0)
        matches = (y == y_hat).int()
        correct = matches.sum()
        tot = torch.prod(torch.tensor(matches.shape))
        


        # dummy metrics
        metrics_dict = {"val_loss": loss.item(), "val_EM": (correct/tot).item(), "val_F1": 0.9}
        self.validation_step_outputs.append(metrics_dict)
        return metrics_dict



    def on_validation_epoch_end(self):
        results = {
            "loss": torch.tensor(
                [x["val_loss"] for x in self.validation_step_outputs]
            ).mean(),
            "EM": torch.tensor(
                [x["val_EM"] for x in self.validation_step_outputs]
            ).mean(),
            "F1": torch.tensor(
                [x["val_F1"] for x in self.validation_step_outputs]
            ).mean(),
        }
        for k, v in results.items():
            self.log(
                f"val_{k}", v, on_epoch=True, prog_bar=True, logger=True, sync_dist=True
            )
            # self.log(f"val_{k}", v, on_epoch=True, prog_bar=True) # , logger=True)

    def test_step(self, batch, batch_idx):
        # x = batch["x"]
        
        
        y = batch["labels"]
        y_hat = self(**batch)
        
        # calculating exact matches 
        y_hat = F.softmax(y_hat.logits, dim=-1)
        a = y_hat.argmax(1)
        y_hat = torch.zeros(y_hat.shape).scatter(1, a.unsqueeze(1), 1.0)
        y_onehot = F.one_hot(y, num_classes=VOCAB_SIZE)
        y = y_onehot.float()
        matches = (y == y_hat).int()
        correct = matches.sum()
        tot = torch.prod(torch.tensor(matches.shape))
        
        
        metrics_dict = {
            "test_EM": (correct/tot).item(),
            "test_F1": 0.8,
        }
        self.test_step_outputs.append(metrics_dict)
        return metrics_dict

    def on_test_epoch_end(self):
        results = {
            "F1": torch.tensor([x["test_EM"] for x in self.test_step_outputs]).mean(),
            "EM": torch.tensor([x["test_F1"] for x in self.test_step_outputs]).mean(),
        }

        for k, v in results.items():
            self.log(
                f"test_{k}",
                v,
                on_epoch=True,
                prog_bar=True,
                logger=True,
                sync_dist=True,
            )

    def configure_optimizers(self):

        # special scheduler for transformers
        optimizer = optim.AdamW(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr=0.001,  # self.cfg_fitting.learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08,
            weight_decay=0.05,
        )
        return {
            "optimizer": optimizer,
            "monitor": "val_loss",
        }

In [2]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pathlib import Path
from dataclasses import dataclass
from pathlib import Path
from babl.data import TextDataset, TextDataModule
from babl.utils import CallbackCollection

class Fitter:
    def __init__(
        self,
        model,
        tokenizer,
        model_name,
        data_path="../inputs",
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.model_name = model_name
        self.data_path = data_path

        ####################################################################################
        @dataclass
        class FittingArgs:
            es_patience: int = 5
            model_dir = Path("/home/nameduser/Code/babl/outputs") / model_name
            max_epoch: int = 10
            fast_dev_run: bool = False

            def __post_init__(self):
                self.model_dir.mkdir(parents=True, exist_ok=True)
                self.model_dir  =  str(self.model_dir)

        ####################################################################################

        self.args = FittingArgs()

    def setup(self):
        data_module = TextDataModule(data_path=self.data_path, tokenizer=self.tokenizer, dev_run=True)

        train_loader = data_module.train_dataloader()
        val_loader = data_module.val_dataloader()
        test_loader = data_module.test_dataloader()

        return train_loader, val_loader, test_loader

    def callbacks(self):
        # cfg_fitting = self.cfg_fitting
        callback_collection = CallbackCollection(self.data_path, self.args)
        return callback_collection()

    def __call__(self):

        logger = TensorBoardLogger(
            save_dir=self.args.model_dir,
            name="lightning_logs",
        )
        Model = self.model
        # get loaders and datamodule to access input shape
        train_loader, val_loader, test_loader = self.setup()
        print("Created training, validating and test loaders .... ")
        # get input shape for onnx exporting
        # input_shape = data_module.input_shape
        # init model
        # kwargs = {}
        # model = Model(**kwargs)

        # setup training, validating and testing routines for the model
        routine = Routine(self.model)

        # Init a trainer to execute routine
        callback_dict = self.callbacks()
        callback_list = [v for (_, v) in callback_dict.items()]
        number_devices = os.getenv("CUDA_VISIBLE_DEVICES", "1,").split(",")
        try:
            number_devices.remove("")
        except ValueError:
            pass

        trainer = Trainer(
            accelerator="cpu",
            devices=len(number_devices),
            # strategy=os.getenv("STRATEGY", "ddp_notebook"),
            sync_batchnorm=True,
            logger=logger,
            max_epochs=self.args.max_epoch,
            callbacks=callback_list,
            num_sanity_val_steps=2,
            # resume_from_checkpoint=self.cfg_fitting.resume_from_checkpoint,
            gradient_clip_val=1.0,
            fast_dev_run=self.args.fast_dev_run,
        )

        trainer.fit(
            routine, train_dataloaders=train_loader, val_dataloaders=val_loader
        )  # ,ckpt_path=PATH)

        if self.args.fast_dev_run:
            # issue with finding best weights path for in fast dev run using last model weights
            model_ckpt_path = callback_dict["checkpoint"].__dict__["last_model_path"]
        else:
            model_ckpt_path = callback_dict["checkpoint"].__dict__["best_model_path"]

        trainer.test(
            dataloaders=test_loader,
            ckpt_path=model_ckpt_path,
        )
        # Return the input_shapes and trainer of the model for exporting
        return trainer

In [ ]:
from babl.models import MODELS_CHOICES, MODELS
from babl.config import T5 as T5Config
from pathlib import Path

model_name = "t5"
full_model_name = MODELS_CHOICES[model_name][0]
t_w_m = MODELS[model_name]

t = t_w_m["tok"]
m = t_w_m["model"]

tokenizer = t.from_pretrained(full_model_name)
model = m.from_pretrained(full_model_name, **T5Config().__dict__)

# placing in training mode 

model.train()

data_path_root = Path("/home/nameduser/Code/babl/inputs")

# # data_path_val = data_path_root / "10k.jsonl"
# # ds = TextDataset(data_path_val, tokenizer=t, plain_text=False)
# # from babl.data import T2TDataCollator
# # from torch.utils.data import DataLoader
# # t_dl = DataLoader(ds, batch_size=64, shuffle=True, collate_fn=T2TDataCollator())
# # test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# # data_module = TextDataModule(data_path, tokenizer)

Fitter(model=model, model_name=full_model_name, tokenizer=tokenizer, data_path=data_path_root)()

In [ ]:
# for b in t_dl:
#     # print(b)
#     m(**b)

import torch.nn.functional as F 


y = torch.tensor([[ 822,   10,  125, 100, 100, 100],
                  [ 822,   10,  116, 100, 100, 100],
                  [ 822,   10,  125, 100, 100, 100],
                  [ 822,   10,  213, 100, 100, 100],
                  [ 822,   10,  213, 100, 100, 100],
                  [ 822,   10,  116, 100, 100, 100],
                  [ 822,   10,  125, 100, 100, 100],
                  [ 822,   10,  116, 100, 100, 100],
                  [ 822,   10,  125, 100, 100, 100],
                  [ 822,   10,  213, 100, 100, 100],
                  [ 822,   10,  213, 100, 100, 100],
                  [ 822,   10,  116, 100, 100, 100]], dtype=torch.long
)
y_hat = torch.tensor([[-20.2879,  -9.8936, -13.5965, -40.7275, -40.8642, -40.8486],
         [-34.0870,  -3.6627, -14.2458,  -46.1296, -46.3147, -46.2990],
         [-30.5974,  -3.4536, -15.5923,  -43.6581, -43.8461, -43.8219],
         [-18.1922,  -8.0767, -14.5352,  -45.5706, -45.7357, -45.7194],
         [-18.1516,  -8.0787, -14.4750,  -45.4796, -45.6429, -45.6272],
         [-18.1262,  -8.1061, -14.4559,  -45.4136, -45.5755, -45.5602],
         [-17.2200,  -9.7170, -14.2499,  -38.4455, -38.5326, -38.4609],
         [-34.3804,  -6.2359, -13.2374,  -42.5014, -42.6473, -42.5558],
         [-27.8060,  -7.1265, -15.4786,  -42.2502, -42.3610, -42.2977],
         [-17.2795,  -7.8251, -15.8752,  -44.6078, -44.7242, -44.6339],
         [-17.1784,  -7.7900, -15.8198,   -44.4029, -44.5184, -44.4275],
         [-17.1213,  -7.7632, -15.7711,   -44.2831, -44.3977, -44.3082]]) 

num_class= 1321
y_hat = torch.stack([y_hat]*num_class, dim=2)

y_hat.shape
y.shape
# y.shape
# y_hat.shape
y = F.one_hot(y, num_classes=1321)
y = y.float()

y.shape == y_hat.shape

y_hat = F.softmax(y_hat, dim=-1)



losses = []
for tok in range(y_hat.shape[1]):
    print(y[:,tok,:].shape)
    print(y_hat[:,tok,:].shape)
    loss = F.cross_entropy(y_hat[:,tok,:] , y[:,tok,:])
    print(loss)
    # loss = F.nll_loss(y_hat[:,tok,:] , y[:,tok,:])
    losses.append(loss)


torch.tensor(losses).mean()
# yx = F.one_hot(y, num_classes=1321)[:,0,:].shape

# loss = F.nll_loss(y_hat , y)


In [ ]:

data_path_root = Path("/home/nameduser/Code/babl/inputs")

data_path_val = data_path_root / "10k.jsonl"
ds = TextDataset(data_path_val, tokenizer=t, plain_text=True )

# from babl.data import T2TDataCollator
# from torch.utils.data import DataLoader
# t_dl = DataLoader(ds, batch_size=64, shuffle=True, collate_fn=T2TDataCollator())
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# data_module = TextDataModule(data_path, tokenizer)


torch

In [ ]:
y = torch.tensor([0,1,1,0,0,1])

yh=  torch.tensor([0,1,1,0,1,1])
(y == yh).int()


torch.prod(torch.tensor(torch.rand((10,10)).shape))


In [ ]:
import torch
torch.manual_seed (1414)

t = torch.randn (8, 4)
a = t.argmax(1)
m = torch.zeros(t.shape).scatter(1, a.unsqueeze(1), 1.0)


print ('\n', t, '\n\n', a, '\n\n', m)

In [ ]:
list({"x": [1, 2, 3, 4]}.values())[0].__len__()